# Récupération des données pour l'étude communale

## 1- Importation des librairies externes et classiques

In [86]:
# Importation des librairies externes
!pip install git+https://github.com/InseeFrLab/Py-Insee-Data@master --ignore-install PyYAML
!pip install python-Levenshtein
!pip install openpyxl
!pip install fuzzywuzzy

  Cloning https://github.com/InseeFrLab/Py-Insee-Data (to revision master) to /tmp/pip-req-build-br46vt1s
  Running command git clone --filter=blob:none -q https://github.com/InseeFrLab/Py-Insee-Data /tmp/pip-req-build-br46vt1s
  Resolved https://github.com/InseeFrLab/Py-Insee-Data to commit 52139da402bc33da6b8892d7d3208f1d2ddfaa02
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached PyYAML-6.0-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (661 kB)
  Using cached requests-2.26.0-py2.py3-none-any.whl (62 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
  Using cached Unidecode-1.3.2-py3-none-any.whl (235 kB)
  Using cached DateTime-4.3-py2.py3-none-any.whl (60 kB)
  Using cached tqdm-4.62.3-py2.py3-none-any.whl (76 kB)
  Using cached pandas-1.3.5-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.5 MB)
  Using cached p

In [87]:
# Importation des librairies classiques de python
import unicodedata
import requests
import time
import os
from tqdm import tqdm
import io

# Importation des librairies de datascience et autres
import pandas as pd
import numpy as np
import pynsee
import pynsee.download
from pynsee.sirene import *
from pynsee.utils.init_conn import init_conn
from zipfile import ZipFile, Path
from fuzzywuzzy import fuzz

In [88]:
# token API insee
init_conn(insee_key=os.environ['INSEE_KEY'], insee_secret=os.environ['INSEE_SECRET'])

## 2- Téléchargement du Code Officiel Géographique

* Pour des raisons de simplicité, nous travaillons en géographie 2021 :

In [138]:
cog_commune = pynsee.download.telechargerDonnees("COG_COMMUNE", date = "dernier")
URL_COG_2021="https://www.insee.fr/fr/statistiques/fichier/5057840/commune2021-csv.zip"

#Télécharger le zip de l'URL
r=requests.get(URL_COG_2021)

open("commune2021-csv.zip", 'wb').write(r.content)
    
    
with ZipFile("commune2021-csv.zip",'r') as myzip:
   data = myzip.open("commune2021.csv")

donnees_cog_2021=pd.read_csv(data,dtype=str)
# On enleve les communes associées,arrondissements et déléguées pour simplifier le dataframe
donnees_cog_2021=donnees_cog_2021[~donnees_cog_2021.TYPECOM.isin(["COMA","COMD","ARM"])]
donnees_cog_2021['NCC et DEP']= donnees_cog_2021['NCC'] + " " + donnees_cog_2021['DEP']
donnees_cog_2021['NCC et COM']= donnees_cog_2021['NCC'] + " ," + donnees_cog_2021['COM']

Data will be stored in the following location: /tmp/tmpd7kcae71


Downloading: : 1.93MiB [00:03, 585kiB/s]                         
/opt/conda/lib/python3.9/site-packages/pynsee/download/__init__.py:182: UserWarning: File in insee.fr modified or corrupted during download
  warnings.warn("File in insee.fr modified or corrupted during download")


No destination directory defined. Data have been written there: /tmp/tmpwki0salq


Extracting: 100%|██████████| 9.00M/9.00M [00:00<00:00, 72.4MB/s]


In [90]:
donnees_cog_2021.describe()

,TYPECOM,COM,REG,DEP,CTCD,ARR,TNCC,NCC,NCCENR,LIBELLE,CAN,COMPARENT
count,34965,34965,34965,34965,34965,34948,34965,34965,34965,34965,34849,0
unique,1,34965,18,101,100,332,8,32488,32553,32707,2039,0
top,COM,01001,44,62,62D,621,0,SAINTE COLOMBE,Sainte-Colombe,Sainte-Colombe,5723,NaN
freq,34965,1,5121,890,890,357,28696,12,12,12,135,NaN


In [139]:
def trouve_commune_with_fuzz(libelle,dep):
    print(libelle,dep)
    mondf=donnees_cog_2021

    mondf['score']=mondf['NCC et DEP'].apply(lambda x: fuzz.token_sort_ratio(x,libelle))
    mondf=mondf.sort_values(by="score",ascending=False)
    return mondf[mondf['DEP']==dep]['NCC et COM'].iloc[0]

In [107]:
donnees_cog_2021

,TYPECOM,COM,REG,DEP,CTCD,ARR,TNCC,NCC,NCCENR,LIBELLE,CAN,COMPARENT,score
0,COM,01001,84,01,01D,012,5,ABERGEMENT CLEMENCIAT,Abergement-Clémenciat,L'Abergement-Clémenciat,0108,NaN,39
1,COM,01002,84,01,01D,011,5,ABERGEMENT DE VAREY,Abergement-de-Varey,L'Abergement-de-Varey,0101,NaN,48
2,COM,01004,84,01,01D,011,1,AMBERIEU EN BUGEY,Ambérieu-en-Bugey,Ambérieu-en-Bugey,0101,NaN,35
3,COM,01005,84,01,01D,012,1,AMBERIEUX EN DOMBES,Ambérieux-en-Dombes,Ambérieux-en-Dombes,0122,NaN,37
4,COM,01006,84,01,01D,011,1,AMBLEON,Ambléon,Ambléon,0104,NaN,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...
37737,COM,97613,06,976,976D,NaN,0,M TSANGAMOUJI,M'Tsangamouji,M'Tsangamouji,97613,NaN,25
37738,COM,97614,06,976,976D,NaN,1,OUANGANI,Ouangani,Ouangani,97610,NaN,19
37739,COM,97615,06,976,976D,NaN,0,PAMANDZI,Pamandzi,Pamandzi,97611,NaN,28
37740,COM,97616,06,976,976D,NaN,0,SADA,Sada,Sada,97612,NaN,15


## 3- Téléchargement des données de SIRENE

In [92]:
### Téléchargement des données Sirene sur les établissements Siret de boucherie-charcuterie (code 4722Z) à l'instant T
# Téléchargement via Insee sirene data  by sirene.fr website

session = requests.session()
URL_ACCUEIL = "https://www.sirene.fr/sirene/public/creation-fichier"
URL_REQUETE_POST = "https://www.sirene.fr/sirene/public/verifier-creation-fichier.action"
URL_DEVIS="https://www.sirene.fr/sirene/client//sirene/client/devis-demande.action"

URL_RECAP="https://www.sirene.fr/sirene/client/telechargement-pret-fichier.action"
URL_TELECHARGEMENT_POST="https://www.sirene.fr/sirene/client/telechargement-fichier"
POST_DATA={
'fichier.caracteristiques.etat.actif': 'true',
'__checkbox_fichier.caracteristiques.etat.actif': 'true',
'__checkbox_fichier.caracteristiques.etat.ferme': 'true',
'fichier.caracteristiques.typeEtablissement': '1',
'fichier.caracteristiques.geoFile': 'false',
'fichier.caracteristiques.cjFile': 'false',
'fichier.caracteristiques.apeFile': 'false',
'fichier.caracteristiques.sirenSiretFile': 'false',
'fichiersCriteres.criteresGeographie.upload': '(binaire)',
'__multiselect_fichier.criteres.geographie.region.modalites': '',
'__multiselect_fichier.criteres.geographie.departement.modalites': '',
'fichiersCriteres.criteresActivite.upload': '(binaire)',
'__multiselect_fichier.criteres.activite.section.modalites': '',
'__multiselect_fichier.criteres.activite.division.modalites': '',
'__multiselect_fichier.criteres.activite.groupe.modalites': '',
'fichier.criteres.activite.sousClasse.modalites': '4722Z',
'__multiselect_fichier.criteres.activite.sousClasse.modalites': '',
'fichiersCriteres.criteresNature.upload': '(binaire)',
'__multiselect_fichier.criteres.nature.entrepriseNiveau1.modalites': '',
'__multiselect_fichier.criteres.nature.entrepriseNiveau2.modalites': '',
'__multiselect_fichier.criteres.nature.entrepriseNiveau3.modalites': '',
'__multiselect_fichier.criteres.trancheEffectif.modalites': '',
'fichier.criteres.dateCreation.date': '',
'fichier.criteres.dateCreation.dateFin': '',
'fichier.criteres.dateMaj.date': '',
'fichier.criteres.dateMaj.dateFin': '',
'fichiersCriteres.criteresSirenSiret.upload': '(binaire)',
'__multiselect_fichier.criteres.criteresCatEnt.modalites': '',
'strutsLocale': 'fr_FR',
}

result=session.get(URL_ACCUEIL)
result=session.post(URL_REQUETE_POST, data=POST_DATA)


result=session.get(URL_RECAP)

result=session.get(URL_DEVIS)

result=session.post(URL_TELECHARGEMENT_POST)

donnees_sirene_fr=pd.read_csv(io.StringIO(result.content.decode('utf-8')),dtype=str)


* les données de Sirene.fr sont des données d'entreprises France entière. Pour l'étude, le nombre d'établissement par commune est considéré :

In [93]:
donnees_sirene_fr_agregees_par_com = donnees_sirene_fr.groupby("codeCommuneEtablissement").size().reset_index(name='NbBoucheriesParCom').set_index("codeCommuneEtablissement")
donnees_sirene_fr_agregees_par_com.describe()

,NbBoucheriesParCom
count,7026.000000
mean,2.860945
std,6.454518
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,179.000000


In [94]:
donnees_sirene_fr_agregees_par_com

,NbBoucheriesParCom
codeCommuneEtablissement,
01004,7
01021,1
01024,1
01025,3
01026,1
...,...
97610,1
97611,4
97615,1


## 3- Téléchargement des données du ministère de l'intérieur

In [134]:
#Téléchargement des données de crimes et délits rapportés déclarés auprès des commissariats 
url_datagouv_min_interieur="https://www.data.gouv.fr/fr/datasets/r/d792092f-b1f7-4180-a367-d043200c1520"

donnees_police_nationale_2018=pd.read_excel(url_datagouv_min_interieur, sheet_name='Services PN 2018')
donnees_gendarmerie_nationale_2018=pd.read_excel(url_datagouv_min_interieur, sheet_name='Services GN 2018' )

del donnees_police_nationale_2018[donnees_police_nationale_2018.columns[0]]
del donnees_gendarmerie_nationale_2018[donnees_gendarmerie_nationale_2018.columns[0]]

url_localisation_pn="https://www.data.gouv.fr/fr/datasets/r/2cb2f356-42b2-4195-a35c-d4e4d986c62b"
url_localisation_gn="https://www.data.gouv.fr/fr/datasets/r/d6a43ef2-d302-4456-90e9-ff2c47cac562"
donnees_localisation_pn=pd.read_csv(url_localisation_pn,sep=';')
donnees_localisation_gn=pd.read_csv(url_localisation_gn,sep=';')



In [135]:
normalMap = {'À': 'A', 'Á': 'A', 'Â': 'A', 'Ã': 'A', 'Ä': 'A',
             'à': 'a', 'á': 'a', 'â': 'a', 'ã': 'a', 'ä': 'a', 'ª': 'A',
             'È': 'E', 'É': 'E', 'Ê': 'E', 'Ë': 'E',
             'è': 'e', 'é': 'e', 'ê': 'e', 'ë': 'e',
             'Í': 'I', 'Ì': 'I', 'Î': 'I', 'Ï': 'I',
             'í': 'i', 'ì': 'i', 'î': 'i', 'ï': 'i',
             'Ò': 'O', 'Ó': 'O', 'Ô': 'O', 'Õ': 'O', 'Ö': 'O',
             'ò': 'o', 'ó': 'o', 'ô': 'o', 'õ': 'o', 'ö': 'o', 'º': 'O',
             'Ù': 'U', 'Ú': 'U', 'Û': 'U', 'Ü': 'U',
             'ù': 'u', 'ú': 'u', 'û': 'u', 'ü': 'u',
             'Ñ': 'N', 'ñ': 'n',
             'Ç': 'C', 'ç': 'c',
             '§': 'S',  '³': '3', '²': '2', '¹': '1',
             'œ':'oe', 'Œ':'OE'}
normalize = str.maketrans(normalMap)
donnees_localisation_pn["service_sans_accent_upper"]=donnees_localisation_pn.service.apply(lambda x : x.translate(normalize).upper())
donnees_localisation_gn["service_sans_accent_upper"]=donnees_localisation_gn.service.apply(lambda x : x.translate(normalize).upper())

## Traitement des données de la Police Nationale

In [140]:
donnees_police_nationale_2018_travail=donnees_police_nationale_2018.T
donnees_police_nationale_2018_travail.columns = donnees_police_nationale_2018_travail.iloc[0]
donnees_police_nationale_2018_travail=donnees_police_nationale_2018_travail.iloc[1: , :]

donnees_police_nationale_2018_travail['Localisation']=donnees_police_nationale_2018_travail.index+" "+donnees_police_nationale_2018_travail['Périmètres'] +" "+donnees_police_nationale_2018_travail['Libellé index \ CSP']

# On retire les données COM par manque d'autres sources pour les autres données
donnees_police_nationale_2018_travail=donnees_police_nationale_2018_travail[donnees_police_nationale_2018_travail.index.str[:2]!='98']

# 1 - Traitements des commissariats

donnees_police_nationale_2018_commissariat_seulement=donnees_police_nationale_2018_travail[donnees_police_nationale_2018_travail['Libellé index \ CSP'].str.contains("CIAT")]



# takes 2m30 to compute / not good matching results for DOM/COM
donnees_police_nationale_2018_commissariat_seulement["DEP"]=np.where(donnees_police_nationale_2018_commissariat_seulement.index.str[:2]=='97', donnees_police_nationale_2018_commissariat_seulement.index.str[:3],donnees_police_nationale_2018_commissariat_seulement.index.str[:2])
donnees_police_nationale_2018_commissariat_seulement["COMMUNES"]=donnees_police_nationale_2018_commissariat_seulement.apply(lambda x: trouve_commune_with_fuzz(x.Localisation, x.DEP), axis=1)
donnees_police_nationale_2018_commissariat_seulement

/tmp/ipykernel_1010/3036272555.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  donnees_police_nationale_2018_commissariat_seulement["DEP"]=np.where(donnees_police_nationale_2018_commissariat_seulement.index.str[:2]=='97', donnees_police_nationale_2018_commissariat_seulement.index.str[:3],donnees_police_nationale_2018_commissariat_seulement.index.str[:2])


01.2 DCSP CIAT DE BOURG EN BRESSE 01
01.3 DCSP CIAT D'OYONNAX 01
02 DCSP CIAT DE CHATEAU THIERRY 02
02.1 DCSP CIAT DE LAON 02
02.2 DCSP CIAT DE SOISSONS 02
02.3 DCSP CIAT DE ST QUENTIN 02
02.4 DCSP CIAT DE TERGNIER LA FERE 02
03 DCSP CIAT DE MONTLUCON 03
03.1 DCSP CIAT DE MOULINS 03
03.2 DCSP CIAT DE VICHY 03
04 DCSP CIAT DE DIGNE 04
04.1 DCSP CIAT DE MANOSQUE 04
05.2 DCSP CIAT DE BRIANCON 05
05.3 DCSP CIAT DE GAP 05
06.6 DCSP CIAT CENTRAL DE NICE 06
06.7 DCSP CIAT D'ANTIBES 06
06.8 DCSP CIAT DE CAGNES SUR MER 06
06.9 DCSP CIAT DE CANNES 06
06.10 DCSP CIAT DE GRASSE 06
06.11 DCSP CIAT DE MENTON 06
07 DCSP CIAT D'AUBENAS 07
07.1 DCSP CIAT DE GUILHERAND GRANGES 07
07.2 DCSP CIAT DE PRIVAS 07
08 DCSP CIAT DE CHARLEVILLE MEZIERES 08
08.1 DCSP CIAT DE SEDAN 08
09 DCSP CIAT DE FOIX 09
09.1 DCSP CIAT DE PAMIERS 09
10 DCSP CIAT CENTRAL DE TROYES 10
11.2 DCSP CIAT DE CARCASSONNE 11
11.3 DCSP CIAT DE NARBONNE 11
12 DCSP CIAT DE DECAZEVILLE 12
12.1 DCSP CIAT DE MILLAU 12
12.2 DCSP CIAT DE RODEZ 1

/tmp/ipykernel_1010/3036272555.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  donnees_police_nationale_2018_commissariat_seulement["COMMUNES"]=donnees_police_nationale_2018_commissariat_seulement.apply(lambda x: trouve_commune_with_fuzz(x.Localisation, x.DEP), axis=1)


Départements,Périmètres,Libellé index \ CSP,Règlements de compte entre malfaireurs,Homicides pour voler et à l'occasion de vols,Homicides pour d'autres motifs,Tentatives d'homicides pour voler et à l'occasion de vols,Tentatives homicides pour d'autres motifs,Coups et blessures volontaires suivis de mort,Autres coups et blessures volontaires criminels ou correctionnels,Prises d'otages à l'occasion de vols,...,"Prix illicittes, publicité fausse et infractions aux règles de la concurrence",Achats et ventes sans factures,Infractions à l'exercice d'une profession règlementée,Infractions au droit de l'urbanisme et de la construction,Fraudes fiscales,Autres délits économiques et financiers,Autres délits,Localisation,DEP,COMMUNES
01.2,DCSP,CIAT DE BOURG EN BRESSE,0,0,2,0,1,1,236,0,...,0,0,1,1,0,3,223,01.2 DCSP CIAT DE BOURG EN BRESSE,01,"BOURG EN BRESSE ,01053"
01.3,DCSP,CIAT D'OYONNAX,0,0,1,0,0,0,135,0,...,0,0,1,1,0,0,64,01.3 DCSP CIAT D'OYONNAX,01,"OYONNAX ,01283"
02,DCSP,CIAT DE CHATEAU THIERRY,0,0,2,0,0,0,102,0,...,0,0,0,0,0,0,47,02 DCSP CIAT DE CHATEAU THIERRY,02,"CHATEAU THIERRY ,02168"
02.1,DCSP,CIAT DE LAON,0,0,1,0,0,0,193,0,...,0,0,0,0,0,0,85,02.1 DCSP CIAT DE LAON,02,"CHIVRES EN LAONNOIS ,02189"
02.2,DCSP,CIAT DE SOISSONS,0,0,0,0,4,0,223,0,...,0,0,0,2,0,2,106,02.2 DCSP CIAT DE SOISSONS,02,"SOISSONS ,02722"
02.3,DCSP,CIAT DE ST QUENTIN,0,0,1,0,1,1,430,0,...,1,0,4,2,0,2,179,02.3 DCSP CIAT DE ST QUENTIN,02,"SAINT QUENTIN ,02691"
02.4,DCSP,CIAT DE TERGNIER LA FERE,0,0,0,0,1,0,119,0,...,0,0,0,0,0,2,41,02.4 DCSP CIAT DE TERGNIER LA FERE,02,"FERE EN TARDENOIS ,02305"
03,DCSP,CIAT DE MONTLUCON,0,0,1,0,4,0,278,0,...,1,0,0,1,0,1,110,03 DCSP CIAT DE MONTLUCON,03,"MONTLUCON ,03185"
03.1,DCSP,CIAT DE MOULINS,0,0,0,0,1,0,222,0,...,0,0,0,0,0,2,73,03.1 DCSP CIAT DE MOULINS,03,"MOULINS ,03190"
03.2,DCSP,CIAT DE VICHY,0,0,0,0,0,1,229,0,...,0,0,0,3,1,1,108,03.2 DCSP CIAT DE VICHY,03,"CHIRAT L EGLISE ,03077"


In [111]:
donnees_police_nationale_2018_commissariat_seulement[donnees_police_nationale_2018_commissariat_seulement.index.str[:2]!=donnees_police_nationale_2018_commissariat_seulement.COMMUNES.str[:2]]

Départements,Périmètres,Libellé index \ CSP,Règlements de compte entre malfaireurs,Homicides pour voler et à l'occasion de vols,Homicides pour d'autres motifs,Tentatives d'homicides pour voler et à l'occasion de vols,Tentatives homicides pour d'autres motifs,Coups et blessures volontaires suivis de mort,Autres coups et blessures volontaires criminels ou correctionnels,Prises d'otages à l'occasion de vols,...,Index non utilisé,"Prix illicittes, publicité fausse et infractions aux règles de la concurrence",Achats et ventes sans factures,Infractions à l'exercice d'une profession règlementée,Infractions au droit de l'urbanisme et de la construction,Fraudes fiscales,Autres délits économiques et financiers,Autres délits,Localisation,COMMUNES
02.1,DCSP,CIAT DE LAON,0,0,1,0,0,0,193,0,...,0,0,0,0,0,0,0,85,02.1 DCSP CIAT DE LAON,01346
02.2,DCSP,CIAT DE SOISSONS,0,0,0,0,4,0,223,0,...,0,0,0,0,2,0,2,106,02.2 DCSP CIAT DE SOISSONS,21610
02.3,DCSP,CIAT DE ST QUENTIN,0,0,1,0,1,1,430,0,...,0,1,0,4,2,0,2,179,02.3 DCSP CIAT DE ST QUENTIN,33467
02.4,DCSP,CIAT DE TERGNIER LA FERE,0,0,0,0,1,0,119,0,...,0,0,0,0,0,0,2,41,02.4 DCSP CIAT DE TERGNIER LA FERE,24484
03.1,DCSP,CIAT DE MOULINS,0,0,0,0,1,0,222,0,...,0,0,0,0,0,0,2,73,03.1 DCSP CIAT DE MOULINS,61297
03.2,DCSP,CIAT DE VICHY,0,0,0,0,0,1,229,0,...,0,0,0,0,3,1,1,108,03.2 DCSP CIAT DE VICHY,62469
04,DCSP,CIAT DE DIGNE,0,0,0,0,0,0,92,0,...,0,0,0,0,0,0,0,78,04 DCSP CIAT DE DIGNE,46264
05.2,DCSP,CIAT DE BRIANCON,0,0,0,0,0,0,65,0,...,0,1,0,1,1,0,0,41,05.2 DCSP CIAT DE BRIANCON,02112
05.3,DCSP,CIAT DE GAP,0,0,0,0,0,0,216,0,...,0,0,0,1,0,0,1,72,05.3 DCSP CIAT DE GAP,03077
06.6,DCSP,CIAT CENTRAL DE NICE,0,0,3,0,17,1,2217,0,...,0,4,0,12,30,1,125,1006,06.6 DCSP CIAT CENTRAL DE NICE,30246


In [98]:
donnees_police_nationale_2018_autres=donnees_police_nationale_2018_travail[~donnees_police_nationale_2018_travail['Libellé index \ CSP'].str.contains("CIAT")]



In [102]:
pd.set_option('display.max_rows', 400)

# 2 - Traitements de Paris
dpn2018_paris=donnees_police_nationale_2018_autres[donnees_police_nationale_2018_autres.index.str.contains('75')]
dpn2018_paris['COM']="75056"

# Les données 92-93-94-95 sont communales et départementales : à analyser finement
dpn2018_92_93_94_95=donnees_police_nationale_2018_autres[
    donnees_police_nationale_2018_autres.index.str.contains('92')
    |donnees_police_nationale_2018_autres.index.str.contains('93')
    |donnees_police_nationale_2018_autres.index.str.contains('94')
    |donnees_police_nationale_2018_autres.index.str.contains('95')
]
dpn2018_92_93_94_95["COMMUNES"]=dpn2018_92_93_94_95['Libellé index \ CSP'].apply(trouve_commune_with_fuzz)
dpn2018_92_93_94_95


/tmp/ipykernel_1010/713157549.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dpn2018_paris['COM']="75056"
/tmp/ipykernel_1010/713157549.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dpn2018_92_93_94_95["COMMUNES"]=dpn2018_92_93_94_95['Libellé index \ CSP'].apply(trouve_commune_with_fuzz)


Départements,Périmètres,Libellé index \ CSP,Règlements de compte entre malfaireurs,Homicides pour voler et à l'occasion de vols,Homicides pour d'autres motifs,Tentatives d'homicides pour voler et à l'occasion de vols,Tentatives homicides pour d'autres motifs,Coups et blessures volontaires suivis de mort,Autres coups et blessures volontaires criminels ou correctionnels,Prises d'otages à l'occasion de vols,...,Index non utilisé,"Prix illicittes, publicité fausse et infractions aux règles de la concurrence",Achats et ventes sans factures,Infractions à l'exercice d'une profession règlementée,Infractions au droit de l'urbanisme et de la construction,Fraudes fiscales,Autres délits économiques et financiers,Autres délits,Localisation,COMMUNES
92,DCCRS,CRS AUTO OUEST IDF VAUCRESSON,0,0,0,0,0,0,7,0,...,0,1,0,0,0,0,0,33,92 DCCRS CRS AUTO OUEST IDF VAUCRESSON,80777
92.1,DCPJ,BNLCOC,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,92.1 DCPJ BNLCOC,64134
92.2,DCPJ,OCBC DCPJ5,0,0,0,0,0,0,0,0,...,0,0,0,4,0,0,3,2,92.2 DCPJ OCBC DCPJ5,51435
92.3,DCPJ,OCLCIFF,0,0,0,0,0,0,0,0,...,0,0,0,1,0,30,41,16,92.3 DCPJ OCLCIFF,50276
92.4,DCPJ,OCLCTIC,0,0,1,0,0,0,4,0,...,0,0,0,0,0,0,0,61,92.4 DCPJ OCLCTIC,2B184
92.5,DCPJ,OCRFM DCPJ10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,92.5 DCPJ OCRFM DCPJ10,89334
92.6,DCPJ,OCRGDF DCPJ9,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,16,6,92.6 DCPJ OCRGDF DCPJ9,26106
92.7,DCPJ,OCRTEH,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,7,6,92.7 DCPJ OCRTEH,27193
92.8,DCPJ,OCRTIS,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,104,22,92.8 DCPJ OCRTIS,05045
92.9,DCPJ,OCRVP,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,2,92.9 DCPJ OCRVP,25436


In [ ]:
dpn2018_hors_paris=donnees_police_nationale_2018_autres[~donnees_police_nationale_2018_autres.index.str.contains('75')]

dpn2018_hors_paris[~dpn2018_hors_paris['Libellé index \ CSP'].str.contains("BMR")
                                     &~dpn2018_hors_paris['Libellé index \ CSP'].str.contains("BCFZ")
                                    &~dpn2018_hors_paris['Libellé index \ CSP'].str.contains("USG")]

* Agrégation par département, transposition et fusion :

In [ ]:
# donnees_police_nationale_2018_par_dep = donnees_police_nationale_2018_par_dep.sum(axis=1).reset_index("departement").set_index("departement")
# donnees_police_nationale_2018_par_dep.rename(columns={0: "Total des crimes et délits relevés par la Police Nationale"},inplace=True)

## Traitement des données de la Gendarmerie Nationale

In [ ]:
donnees_gendarmerie_nationale_2018_par_com=donnees_gendarmerie_nationale_2018.T
donnees_gendarmerie_nationale_2018_par_com.columns = donnees_gendarmerie_nationale_2018_par_com.iloc[0]
donnees_gendarmerie_nationale_2018_par_com=donnees_gendarmerie_nationale_2018_par_com.iloc[1: , :]
donnees_gendarmerie_nationale_2018_par_com.columns


# On retire les données COM par manque d'autres sources pour les autres données
donnees_gendarmerie_nationale_2018_par_com=donnees_gendarmerie_nationale_2018_par_com[donnees_gendarmerie_nationale_2018_par_com.index.str[:2]!='98']

#Take 2 minutes to compute

donnees_gendarmerie_nationale_2018_par_com["COMMUNES"]=donnees_gendarmerie_nationale_2018_par_com['Libellé index \ CGD'].apply(trouve_commune_with_fuzz)
donnees_gendarmerie_nationale_2018_par_com
# donnees_gendarmerie_nationale_2018_par_dep.columns = donnees_gendarmerie_nationale_2018_par_dep.iloc[0]
# donnees_gendarmerie_nationale_2018_par_dep=donnees_gendarmerie_nationale_2018_par_dep.iloc[1: , :]

# donnees_gendarmerie_nationale_2018_par_dep.drop(donnees_gendarmerie_nationale_2018_par_dep[donnees_gendarmerie_nationale_2018_par_dep.index.str[:2]=='98'].index,inplace=True)
# donnees_gendarmerie_nationale_2018_par_dep["departement"]=np.where(donnees_gendarmerie_nationale_2018_par_dep.index.str[:2]=='97', donnees_gendarmerie_nationale_2018_par_dep.index.str[:3],donnees_gendarmerie_nationale_2018_par_dep.index.str[:2])

# donnees_gendarmerie_nationale_2018_par_dep = donnees_gendarmerie_nationale_2018_par_dep.groupby("departement").sum() 
# donnees_gendarmerie_nationale_2018_par_dep

In [ ]:
donnees_localisation_gn[~donnees_localisation_gn["code_commune_insee"].isin(donnees_localisation_gn["code_commune_insee"].unique())]
donnees_localisation_gn
donnees_localisation_pn



In [ ]:
#50 appels par seconde autorisés
strinput='41 Boulevard Louis Dupuy 01100 OYONNAX'
url_api="https://api-adresse.data.gouv.fr/search/?q="
response=requests.get(url_api+strinput)
print(response.json())

In [ ]:
donnees_gendarmerie_nationale_2018_par_dep = donnees_gendarmerie_nationale_2018_par_dep.sum(axis=1).reset_index("departement").set_index("departement")
donnees_gendarmerie_nationale_2018_par_dep.rename(columns={0: "Total des crimes et délits relevés par la Gendarmerie Nationale"},inplace=True)
donnees_gendarmerie_nationale_2018_par_dep

## Fusion des données de la Police Nationale et de la Gendarmerie Nationale 

In [ ]:
merged_crimes_delits_2018_par_dep = donnees_police_nationale_2018_par_dep.merge(donnees_gendarmerie_nationale_2018_par_dep, on="departement", how='left')
merged_crimes_delits_2018_par_dep

## 4 - Téléchargement des revenus localisés

In [ ]:
URL_FILO_ZIP_2018="https://www.insee.fr/fr/statistiques/fichier/5009236/base-cc-filosofi-2018_CSV_geo2021.zip"
#Télécharger le zip de l'URL
r=requests.get(URL_FILO_ZIP_2018)

open("base-cc-filosofi-2018_CSV_geo2021.zip", 'wb').write(r.content)
    
    
with ZipFile("base-cc-filosofi-2018_CSV_geo2021.zip",'r') as myzip:
    data = myzip.open("cc_filosofi_2018_COM-geo2021.CSV")
    
donnees_filo_communes_2018=pd.read_csv(data,sep=';')       

In [ ]:
donnees_filo_communes_2018

## 5 - Téléchargement des populations légales

In [ ]:
#Téléchargement des populations légales du millénisme 2018 
URL_POP_LEG="https://www.insee.fr/fr/statistiques/fichier/4989724/ensemble.zip"

#Télécharger le zip de l'URL
r=requests.get(URL_POP_LEG)

open("ensemble.zip", 'wb').write(r.content)
    
    
with ZipFile("ensemble.zip",'r') as myzip:
    data = myzip.open("Communes.csv")

donnees_pop_leg_18=pd.read_csv(data,sep=';',dtype=str)
donnees_pop_leg_18["CODE_INSEE"]=donnees_pop_leg_18["CODREG"]+donnees_pop_leg_18["CODCOM"]
donnees_pop_leg_18=donnees_pop_leg_18.set_index("CODE_INSEE")


In [ ]:
donnees_pop_leg_18.sample(10)

In [ ]:
donnees_pop_leg_18.describe()

## 6 - Téléchargement des données d'emplois localisées

In [ ]:
#Téléchargement des données d'emploi localisées
#donnees_emploi_statut_18 = pynsee.download.telechargerDonnees("ESTEL_T201", date = "2018")
#donnees_emploi_statut_18.sample(10)

#donnees_emploi_sexe_18 = pynsee.download.telechargerDonnees("ESTEL_T202", date = "2018")
#donnees_emploi_sexe_18.sample(10)


URL_TAUX_CHOMAGE_PAR_DEP="https://www.insee.fr/fr/statistiques/fichier/2012804/sl_etc_2021T2.xls"


donnees_chomage_par_departement_series_longues=pd.read_excel(URL_TAUX_CHOMAGE_PAR_DEP, sheet_name='Département', skiprows=3)

donnees_chomage_par_departement_series_longues=donnees_chomage_par_departement_series_longues.iloc[:100]
donnees_chomage_par_departement_series_longues=donnees_chomage_par_departement_series_longues.set_index("Code")
donnees_chomage_par_departement_series_longues.index=donnees_chomage_par_departement_series_longues.index.astype(str)

In [ ]:
donnees_chomage_par_departement_series_longues.describe()

## Telechargement des données de répartition de gendarmes et policiers en 2019 sur le territoire (au niveau départemental)

In [ ]:
URL_NB_PN_GN_DEP = "https://www.insee.fr/fr/statistiques/fichier/5763601/IREF_SECUR21-F29.xlsx"
donnees_nb_pn_gn_dep=pd.read_excel(URL_NB_PN_GN_DEP, sheet_name='Figure 3', skiprows=1)
donnees_nb_pn_gn_dep = donnees_nb_pn_gn_dep.set_index("Unnamed: 0")
donnees_nb_pn_gn_dep=donnees_nb_pn_gn_dep.iloc[:100]
donnees_nb_pn_gn_dep=donnees_nb_pn_gn_dep.rename(columns={"Unnamed: 0": "CODGEO", "Unnamed: 1": "Département", "Taux pour 10 000 habitants":"Nb_PN_GN_dep_100k_hab"}, index={'Unnamed:0': 'CODDEP'})
donnees_nb_pn_gn_dep.index = donnees_nb_pn_gn_dep.index.astype(str)
donnees_nb_pn_gn_dep.index.names = ['CODDEP']
donnees_nb_pn_gn_dep


## 8 - Fusion des données

In [ ]:
donnees_2018=merged_crimes_delits_2018_par_dep.merge(donnees_filo_departement_2018,left_index=True,right_index=True)
donnees_2018=donnees_2018.merge(donnees_pop_leg_18,left_index=True,right_index=True)
donnees_2018=donnees_2018.merge(donnees_sirene_fr_agregees_par_departement,left_index=True,right_index=True)
donnees_2018=donnees_2018.merge(donnees_chomage_par_departement_series_longues,left_index=True,right_index=True)
donnees_2018=donnees_2018.merge(donnees_nb_pn_gn_dep,left_index=True,right_index=True)
donnees_2018=donnees_2018[['REG', 'Libellé', 'Total des crimes et délits relevés par la Police Nationale', 'Total des crimes et délits relevés par la Gendarmerie Nationale', 'MED18' , 'TP6018', 'D118', 'D918', 'RD18', 'PTOT','T1_2018', 'NbBoucheriesParDep', 'Nb_PN_GN_dep_100k_hab']]
donnees_2018

* On se restreint au champ métropolitain car les données d'outre mer ne sont pas exhaustives sur toutes nos sources 

In [ ]:
donnees_2018 = donnees_2018[donnees_2018.index.str[:2]!='97']
donnees_2018
donnees_2018.describe()

* On fait l'hypothèse d'unicité du dépôt de plainte ou de signalement en gardarmerie ou bien en commissariat, ce qui permet de sommer les crimes et délits au niveau départemental

In [ ]:
donnees_2018['Total des crimes et délits relevés par la PN et GN'] = donnees_2018[['Total des crimes et délits relevés par la Police Nationale', 'Total des crimes et délits relevés par la Gendarmerie Nationale']].sum(axis=1)

In [ ]:
donnees_2018 = donnees_2018.rename(columns={"Total des crimes et délits relevés par la Police Nationale": "Crim_Del_PN", "Total des crimes et délits relevés par la Gendarmerie Nationale": "Crim_Del_GN", "Total des crimes et délits relevés par la PN et GN": "Crim_Del_PN_GN"})
donnees_2018

## 9 - On créé une base de données rapportant le nombre de crimes et délits, et le nombre de boucheries pour 100 000 habitants 


In [ ]:
donnees_2018_hab = donnees_2018
donnees_2018_hab['Nb_Boucherie_dep_hab']=round(donnees_2018_hab['NbBoucheriesParDep']/donnees_2018_hab['PTOT']*100000)
donnees_2018_hab['Crim_Del_GN_hab']=round(donnees_2018_hab['Crim_Del_GN']/donnees_2018_hab['PTOT']*100000)
donnees_2018_hab['Crim_Del_PN_hab']=round(donnees_2018_hab['Crim_Del_PN']/donnees_2018_hab['PTOT']*100000)
donnees_2018_hab['Crim_Del_PN_GN_hab']=round(donnees_2018_hab['Crim_Del_PN_GN']/donnees_2018_hab['PTOT']*100000)
donnees_2018_hab.drop(['Crim_Del_PN', 'Crim_Del_GN', 'PTOT', 'NbBoucheriesParDep'], axis=1, inplace = True)
donnees_2018_hab


In [ ]:
%store donnees_2018
%store donnees_2018_hab

In [ ]:
import re
from bs4 import BeautifulSoup
import time
headers = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'
}

url="https://www.ville-ideale.fr/villespardepts.php"
url_cherche="https://www.ville-ideale.fr/scripts/cherche.php"
base_url="https://www.ville-ideale.fr"
s = requests.session()
r=s.get(url,headers=headers)
print(r)
print(r.text)
time.sleep(1)

liste_departement=re.findall(r'affdept\(\'(\d+)\',',r.text)
for _, numdep in enumerate(liste_departement):
    r=s.post(url_cherche,data={'dept': numdep},headers=headers)
    print(r)
    soup = BeautifulSoup(r.text, 'html.parser')
    for link in soup.find_all('a'):
        time.sleep(1)
        print(link.get('href'))
        r=s.get(base_url+link.get('href'),headers=headers)
        print(r)
        soup = BeautifulSoup(r.text, 'html.parser')
        print(soup.find(id="tablonotes").findAll("tr"))
        print(type(soup.find(id="tablonotes").findAll("tr")))
        liste_tr_securite=[tr.text for tr in list(soup.find(id="tablonotes").findAll("tr")) if "Sécurité" in tr.text ]
        if liste_tr_securite:
            note_securite=re.findall(r'\d+', liste_tr_securite[0])
            print('.'.join(note_securite))
        break
    break

In [ ]:
!pip install lxml
from lxml.html import fromstring
import requests
from itertools import cycle
import traceback
def get_proxies():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = set()
    for i in parser.xpath('//tbody/tr'):
        if i.xpath('.//td[7][contains(text(),"yes")]') and  i.xpath('.//td[5][contains(text(),"elite proxy")]'):
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            proxies.add(proxy)
    return proxies

proxies = get_proxies()
print(proxies)
proxy_pool = cycle(proxies)
url = 'https://www.ville-ideale.fr'

bool_continue=True
i=0
while bool_continue and i <10:
#Get a proxy from the pool
    i+=1
    proxy = next(proxy_pool)
    print("Request #%d"%i)
    try:
        s = requests.Session()
        s.proxies = {"http": proxy, "https": proxy}
        response = s.get(url)
        print(response.json())
        bool_continue=False
    except:
        #Most free proxies will often get connection errors. You will have retry the entire request using another proxy to work. 
        #We will just skip retries as its beyond the scope of this tutorial and we are only downloading a single url 
        print("Skipping. Connnection error")